In [6]:
# Use the same AWS Bedrock setup that works in your other notebook
from langchain.chat_models import init_chat_model
from langchain.agents import AgentType, initialize_agent
from langchain.tools import BaseTool
from langchain.pydantic_v1 import BaseModel, Field
from typing import Type
import pandas as pd
import getout_of_text_3 as got3

# Initialize the Bedrock model (same as your working notebook)
model_id = 'openai.gpt-oss-120b-1:0'  # This model supports tools
llm = init_chat_model(
    model_id, 
    model_provider="bedrock_converse",
    credentials_profile_name='atn-developer'
)

print(f"Model initialized: {model_id}")
print("Ready to create custom tools and agents!")

/Users/ejacquot/Documents/Github/getout_of_text_3/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


Model initialized: openai.gpt-oss-120b-1:0
Ready to create custom tools and agents!


In [7]:
# Let's create a simple test tool to verify the agent setup works
class SimpleTestInput(BaseModel):
    """Input for simple test tool."""
    message: str = Field(description="A message to echo back")

class SimpleTestTool(BaseTool):
    """A simple test tool that echoes back a message."""
    
    name: str = "simple_test"
    description: str = "A simple tool that echoes back any message you send to it."
    args_schema: Type[BaseModel] = SimpleTestInput
    
    def _run(self, message: str) -> str:
        """Echo back the message."""
        return f"Tool received and echoing back: {message}"
    
    async def _arun(self, message: str) -> str:
        """Async version."""
        return self._run(message)

# Create the test tool
test_tool = SimpleTestTool()

# Test the tool directly
print("Direct tool test:")
result = test_tool._run("Hello from the tool!")
print(result)

Direct tool test:
Tool received and echoing back: Hello from the tool!


In [8]:
# Now create an agent with the test tool using the modern LangGraph approach
from langgraph.prebuilt import create_react_agent

# Create agent with our test tool
tools = [test_tool]
agent_executor = create_react_agent(llm, tools)

print(f"Agent created with tools: {[tool.name for tool in tools]}")
print("Agent is ready to use!")

Agent created with tools: ['simple_test']
Agent is ready to use!


In [9]:
# Test the agent with the simple tool
def test_agent(question: str):
    """Helper function to test the agent."""
    input_message = {"role": "user", "content": question}
    response = agent_executor.invoke({"messages": [input_message]})
    
    print(f"=== QUESTION: {question} ===")
    for message in response["messages"]:
        message.pretty_print()
    print("\n" + "="*60 + "\n")
    
    return response

# Test 1: Simple greeting (should not use tools)
test_agent("Hello! How are you?")

=== QUESTION: Hello! How are you? ===
================================ Human Message =================================

Hello! How are you?
================================== Ai Message ==================================

[{'type': 'reasoning_content', 'reasoning_content': {'text': 'The user says hello. Simple friendly response.', 'signature': ''}}, {'type': 'text', 'text': "Hello! I'm doing great, thanks for asking. How can I assist you today?"}]




{'messages': [HumanMessage(content='Hello! How are you?', additional_kwargs={}, response_metadata={}, id='ce51a1aa-5eba-4ea2-b601-d4509d5312cb'),
  AIMessage(content=[{'type': 'reasoning_content', 'reasoning_content': {'text': 'The user says hello. Simple friendly response.', 'signature': ''}}, {'type': 'text', 'text': "Hello! I'm doing great, thanks for asking. How can I assist you today?"}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '7a3c880c-2186-4627-81ce-2bfcd99d22f3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 03 Oct 2025 05:51:42 GMT', 'content-type': 'application/json', 'content-length': '371', 'connection': 'keep-alive', 'x-amzn-requestid': '7a3c880c-2186-4627-81ce-2bfcd99d22f3'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [672]}, 'model_name': 'openai.gpt-oss-120b-1:0'}, id='run--8c3aa00f-1f03-4086-afba-408554c9f924-0', usage_metadata={'input_tokens': 136, 'output_tokens': 36, 'total_tokens': 172, 'input_

In [10]:
# Test 2: Question that should trigger the tool
test_agent("Can you use your simple test tool to echo back the message 'Agent tool test successful!'?")

=== QUESTION: Can you use your simple test tool to echo back the message 'Agent tool test successful!'? ===
================================ Human Message =================================

Can you use your simple test tool to echo back the message 'Agent tool test successful!'?
================================== Ai Message ==================================

[{'type': 'reasoning_content', 'reasoning_content': {'text': "User asks to use simple test tool to echo back the message. We need to call the function simple_test with message 'Agent tool test successful!'.", 'signature': ''}}, {'type': 'text', 'text': 'Agent tool test successful!'}, {'type': 'tool_use', 'name': 'simple_test', 'input': {'message': 'Agent tool test successful!'}, 'id': 'tooluse_u0-MnwnJR1OxGRMDPwr2Kw'}]
Tool Calls:
  simple_test (tooluse_u0-MnwnJR1OxGRMDPwr2Kw)
 Call ID: tooluse_u0-MnwnJR1OxGRMDPwr2Kw
  Args:
    message: Agent tool test successful!
================================= Tool Message ===================

{'messages': [HumanMessage(content="Can you use your simple test tool to echo back the message 'Agent tool test successful!'?", additional_kwargs={}, response_metadata={}, id='212c89da-133c-4228-8dae-f68118e3d947'),
  AIMessage(content=[{'type': 'reasoning_content', 'reasoning_content': {'text': "User asks to use simple test tool to echo back the message. We need to call the function simple_test with message 'Agent tool test successful!'.", 'signature': ''}}, {'type': 'text', 'text': 'Agent tool test successful!'}, {'type': 'tool_use', 'name': 'simple_test', 'input': {'message': 'Agent tool test successful!'}, 'id': 'tooluse_u0-MnwnJR1OxGRMDPwr2Kw'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '0f8f110c-3a2c-449b-80ff-6ab31cb318d4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 03 Oct 2025 05:51:44 GMT', 'content-type': 'application/json', 'content-length': '556', 'connection': 'keep-alive', 'x-amzn-requestid': '0f8f110c-3a2c-449b-80ff-6ab31cb318d

## Following AWS Lambda Documentation

Adapting the official AWS Lambda tool guide to work with AWS Bedrock instead of OpenAI.

First, let's install the required dependencies:

In [13]:
# Install required dependencies for AWS Lambda tool
%pip install --upgrade --quiet boto3
%pip install --upgrade --quiet langchain-community


[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.Note: you may need to restart the kernel to use updated packages.


[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [14]:
# Try using the official AWS Lambda tool with Bedrock instead of OpenAI
from langchain.agents import AgentType, initialize_agent, load_tools

# Use your existing Bedrock LLM instead of OpenAI
# llm is already defined in cell 1 with AWS Bedrock

try:
    tools = load_tools(
        ["awslambda"],
        awslambda_tool_name="email-sender",
        awslambda_tool_description="sends an email with the specified content to test@testing123.com",
        function_name="testFunction1",
    )
    
    print("Successfully loaded AWS Lambda tools!")
    print(f"Tools loaded: {[tool.name for tool in tools]}")
    
except Exception as e:
    print(f"Error loading tools: {e}")
    print("This might happen if langchain-community is not installed or configured properly")

Error loading tools: You must specify a region.
This might happen if langchain-community is not installed or configured properly


In [15]:
# Create agent with official AWS Lambda tool and Bedrock LLM
try:
    # Try the modern LangGraph approach first
    from langgraph.prebuilt import create_react_agent
    
    official_agent_executor = create_react_agent(llm, tools)
    print("✅ Created agent with official AWS Lambda tool using LangGraph")
    print("This is the modern approach that should work with Bedrock")
    
except Exception as e:
    print(f"LangGraph approach failed: {e}")
    
    # Fallback to the traditional approach
    try:
        agent = initialize_agent(
            tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
        )
        print("✅ Created agent with official AWS Lambda tool using traditional approach")
        print("Note: This might not work as well with Bedrock models")
        
    except Exception as e2:
        print(f"Traditional approach also failed: {e2}")
        print("The official tool might not be compatible with Bedrock models")

✅ Created agent with official AWS Lambda tool using LangGraph
This is the modern approach that should work with Bedrock


In [16]:
# Test the official AWS Lambda tool with Bedrock
def test_official_lambda_agent(question: str):
    """Test the official AWS Lambda tool with Bedrock."""
    try:
        # Use the modern LangGraph agent if available
        if 'official_agent_executor' in globals():
            input_message = {"role": "user", "content": question}
            response = official_agent_executor.invoke({"messages": [input_message]})
            
            print(f"=== QUESTION: {question} ===")
            for message in response["messages"]:
                message.pretty_print()
            
        # Fallback to traditional agent
        elif 'agent' in globals():
            response = agent.run(question)
            print(f"=== QUESTION: {question} ===")
            print(f"RESPONSE: {response}")
            
        else:
            print("No agent available to test")
            
    except Exception as e:
        print(f"Error testing official Lambda tool: {e}")

# Test the official tool approach
print("=== TESTING OFFICIAL AWS LAMBDA TOOL WITH BEDROCK ===")
test_official_lambda_agent("Send an email to test@testing123.com saying 'Hello from official AWS Lambda tool with Bedrock!'.")

=== TESTING OFFICIAL AWS LAMBDA TOOL WITH BEDROCK ===
=== QUESTION: Send an email to test@testing123.com saying 'Hello from official AWS Lambda tool with Bedrock!'. ===
================================ Human Message =================================

Send an email to test@testing123.com saying 'Hello from official AWS Lambda tool with Bedrock!'.
================================== Ai Message ==================================

[{'type': 'reasoning_content', 'reasoning_content': {'text': 'The user wants to "Send an email to test@testing123.com saying \'Hello from official AWS Lambda tool with Bedrock!\'."\n\nWe are ChatGPT with no direct email-sending capability. We can respond that we cannot send email. Or we could ask for more detail. The request is to send an email; we cannot perform that. We can provide instructions on how to send email via AWS Lambda or via other tool. So respond accordingly.', 'signature': ''}}, {'type': 'text', 'text': 'I’m not able to send emails directly from he

In [25]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_aws import BedrockLLM, ChatBedrock


# Option 1: BedrockLLM (completion style)
try:
    llm_completion = BedrockLLM(
        model_id = 'openai.gpt-oss-120b-1:0',  # Specify model
        region_name="us-east-1",
        credentials_profile_name="atn-developer"
    )
    print("✅ BedrockLLM (completion) initialized successfully")
except Exception as e:
    print(f"❌ BedrockLLM failed: {e}")

# Option 2: ChatBedrock (chat style - usually better for agents)
try:
    llm_chat = ChatBedrock(
        model_id = 'openai.gpt-oss-120b-1:0',
        region_name="us-east-1",
        credentials_profile_name="atn-developer"
    )
    print("✅ ChatBedrock initialized successfully")
except Exception as e:
    print(f"❌ ChatBedrock failed: {e}")

# Option 3: Your working approach from cell 1
print("✅ Your working init_chat_model approach is already available as 'llm'")

print("\nTesting which LLM to use for the agent...")

✅ BedrockLLM (completion) initialized successfully
✅ ChatBedrock initialized successfully
✅ Your working init_chat_model approach is already available as 'llm'

Testing which LLM to use for the agent...


In [26]:
# Test each LLM approach with the AWS Lambda tool
tools = load_tools(
    ["awslambda"],
    awslambda_tool_name="email-sender",
    awslambda_tool_description="sends an email with the specified content to test@testing123.com",
    function_name="testFunction1",
)

print("Testing different Bedrock LLM approaches:")

# Test 1: BedrockLLM (completion style)
try:
    agent_completion = initialize_agent(
        tools, llm_completion, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
    )
    print("✅ Agent created with BedrockLLM (completion)")
    
    # Test it
    result = agent_completion.run("Send an email to test@testing123.com saying 'Hello from BedrockLLM!'")
    print(f"BedrockLLM result: {result}")
    
except Exception as e:
    print(f"❌ BedrockLLM agent failed: {e}")

print("\n" + "="*60 + "\n")

NoRegionError: You must specify a region.

In [27]:
# Test 2: ChatBedrock (chat style)
try:
    agent_chat = initialize_agent(
        tools, llm_chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
    )
    print("✅ Agent created with ChatBedrock")
    
    # Test it
    result = agent_chat.run("Send an email to test@testing123.com saying 'Hello from ChatBedrock!'")
    print(f"ChatBedrock result: {result}")
    
except Exception as e:
    print(f"❌ ChatBedrock agent failed: {e}")

print("\n" + "="*60 + "\n")

❌ ChatBedrock agent failed: type object 'SimpleTestInput' has no attribute 'model_json_schema'




In [28]:
# Test 3: Your working init_chat_model approach (for comparison)
try:
    agent_working = initialize_agent(
        tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
    )
    print("✅ Agent created with init_chat_model (your working approach)")
    
    # Test it
    result = agent_working.run("Send an email to test@testing123.com saying 'Hello from init_chat_model!'")
    print(f"init_chat_model result: {result}")
    
except Exception as e:
    print(f"❌ init_chat_model agent failed: {e}")

print("\n" + "="*60 + "\n")
print("COMPARISON COMPLETE - Check which approach worked best!")

❌ init_chat_model agent failed: type object 'SimpleTestInput' has no attribute 'model_json_schema'


COMPARISON COMPLETE - Check which approach worked best!


In [37]:

from langchain.agents import AgentType, initialize_agent, load_tools
#from langchain_openai import OpenAI
from langchain_aws import BedrockLLM

llm = BedrockLLM(
    model_id='openai.gpt-oss-120b-1:0',  # Specify model
    region_name="us-east-1",
    credentials_profile_name="atn-developer"
)

tools = load_tools(
    ["awslambda"],
    awslambda_tool_name="email-sender",
    awslambda_tool_description="sends an email with the specified content to jacquot.etienne@gmail.com",
    function_name="testFunction1",
    region="us-east-1"
)


agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

agent.run("Send an email to jacquot.etienne@gmail.com saying hello world.")

NoRegionError: You must specify a region.

## Comparison: Official Tool vs Custom Tool

Now you have both approaches implemented:

1. **Official AWS Lambda Tool** (cells above): Uses `load_tools(["awslambda"])` from langchain-community
2. **Custom AWS Lambda Tool** (cells 7-10): Our custom implementation

### Key Differences:

**Official Tool:**
- ✅ Maintained by LangChain team
- ✅ Standard interface
- ❓ May have compatibility issues with Bedrock models
- ❓ Less control over error handling and payload format

**Custom Tool:**
- ✅ Full control over implementation
- ✅ Designed specifically for your AWS Bedrock setup
- ✅ Better error handling and debugging
- ✅ Can customize payload format for your specific Lambda function
- ❌ More code to maintain

### Recommendation:
Try the official tool first (above cells). If it works well with your Bedrock setup, use it. If you encounter issues or need more control, use the custom tool approach (cells 7-10).

In [12]:
# Custom AWS Lambda tool that works with AWS Bedrock
import boto3
from langchain.tools import BaseTool
from langchain.pydantic_v1 import BaseModel, Field
from typing import Type
import json

class LambdaEmailInput(BaseModel):
    """Input for Lambda email tool."""
    email_content: str = Field(description="The content of the email to send")
    recipient: str = Field(default="test@testing123.com", description="Email recipient")

class AWSLambdaEmailTool(BaseTool):
    """Custom tool for sending emails via AWS Lambda."""
    
    name: str = "aws_lambda_email"
    description: str = """
    Sends an email using AWS Lambda function. 
    Provide the email content and optionally the recipient email address.
    Default recipient is test@testing123.com.
    """
    args_schema: Type[BaseModel] = LambdaEmailInput
    
    def __init__(self, function_name: str, aws_profile: str = None, **kwargs):
        super().__init__(**kwargs)
        self.function_name = function_name
        
        # Initialize boto3 Lambda client with your AWS profile
        session = boto3.Session(profile_name=aws_profile) if aws_profile else boto3.Session()
        self.lambda_client = session.client('lambda')
    
    def _run(self, email_content: str, recipient: str = "test@testing123.com") -> str:
        """Send email via AWS Lambda."""
        try:
            # Prepare the payload for your Lambda function
            payload = {
                "email_content": email_content,
                "recipient": recipient
            }
            
            # Invoke the Lambda function
            response = self.lambda_client.invoke(
                FunctionName=self.function_name,
                InvocationType='RequestResponse',  # Synchronous
                Payload=json.dumps(payload)
            )
            
            # Parse the response
            response_payload = json.loads(response['Payload'].read())
            
            if response['StatusCode'] == 200:
                return f"Email sent successfully to {recipient}. Lambda response: {response_payload}"
            else:
                return f"Failed to send email. Status: {response['StatusCode']}, Response: {response_payload}"
                
        except Exception as e:
            return f"Error invoking Lambda function: {str(e)}"
    
    async def _arun(self, email_content: str, recipient: str = "test@testing123.com") -> str:
        """Async version - for now just calls the sync version."""
        return self._run(email_content, recipient)

# Create the Lambda email tool with your AWS profile
lambda_email_tool = AWSLambdaEmailTool(
    function_name="testFunction1",
    aws_profile="atn-developer"  # Same profile you use for Bedrock
)

print("Custom AWS Lambda email tool created!")
print(f"Function name: testFunction1")
print(f"AWS profile: atn-developer")

ValueError: "AWSLambdaEmailTool" object has no field "function_name"

In [ ]:
# Create an agent with your AWS Bedrock LLM and custom Lambda tool
from langgraph.prebuilt import create_react_agent

# Use the Bedrock LLM from the first cell and add the Lambda tool
tools = [lambda_email_tool]
agent_executor = create_react_agent(llm, tools)  # llm is your Bedrock model

print(f"Agent created with AWS Bedrock LLM and tools: {[tool.name for tool in tools]}")
print("Ready to send emails via Lambda using AWS Bedrock!")

In [ ]:
# Test the Lambda email tool directly first
print("=== DIRECT TOOL TEST ===")
try:
    result = lambda_email_tool._run(
        email_content="Hello world from AWS Bedrock and Lambda!",
        recipient="test@testing123.com"
    )
    print(result)
except Exception as e:
    print(f"Direct tool test failed: {e}")
    print("Note: This might fail if the Lambda function doesn't exist or isn't configured properly")

In [ ]:
# Test the agent with the Lambda email tool
def test_lambda_agent(question: str):
    """Helper function to test the agent with Lambda."""
    input_message = {"role": "user", "content": question}
    response = agent_executor.invoke({"messages": [input_message]})
    
    print(f"=== QUESTION: {question} ===")
    for message in response["messages"]:
        message.pretty_print()
    print("\n" + "="*60 + "\n")
    
    return response

# Test: Ask the agent to send an email
print("=== AGENT TEST ===")
test_lambda_agent("Send an email to test@testing123.com saying 'Hello world from AWS Bedrock agent!'.")